# Setting optimal MLP's hiperparameters for bike dataset

In [ ]:
# Importing all necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import optuna

import pandas as pd
pd.set_option('display.max_columns', 50)

In [4]:
# Read preprocessed bike dataset
bike_df = pd.read_csv('./prep_bike_data.csv')
bike_df

,rank(season),yr,rank(mnth),rank(hr),holiday,rank(weekday),workingday,weathersit,rank(temp),rank(atemp),rank(hum),rank(windspeed),rank(casual),rank(registered),cnt
0,-1.16483,0,-1.737916,-2.035759,0,1.459083,0,1,-1.276942,-0.946560,0.770810,-1.532339,-0.774893,-1.037755,16
1,-1.16483,0,-1.737916,-1.533273,0,1.459083,0,1,-1.448913,-1.045073,0.734244,-1.532339,-0.356451,-0.699187,40
2,-1.16483,0,-1.737916,-1.259057,0,1.459083,0,1,-1.448913,-1.045073,0.734244,-1.532339,-0.564583,-0.766063,32
3,-1.16483,0,-1.737916,-1.059753,0,1.459083,0,1,-1.276942,-0.946560,0.509186,-1.532339,-0.774893,-1.126601,13
4,-1.16483,0,-1.737916,-0.897469,0,1.459083,0,1,-1.276942,-0.946560,0.509186,-1.532339,-1.690293,-2.448942,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,-1.16483,1,1.720548,0.883425,0,-0.788204,1,2,-1.117398,-1.152974,-0.121890,-0.127559,-0.204644,-0.057581,119
17375,-1.16483,1,1.720548,1.051068,0,-0.788204,1,2,-1.117398,-1.152974,-0.121890,-0.127559,-0.356451,-0.265968,89
17376,-1.16483,1,1.720548,1.255084,0,-0.788204,1,1,-1.117398,-1.152974,-0.121890,-0.127559,-0.417236,-0.250236,90
17377,-1.16483,1,1.720548,1.531407,0,-0.788204,1,1,-1.117398,-1.045073,-0.276442,-0.384024,-0.119057,-0.537970,61


In [5]:
# Separate data into features and targets
X = bike_df.iloc[:, :-1].values
y = bike_df.iloc[:, -1].values
y = y.reshape(-1, 1)

# Data normalization
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y)

# Splitting data into train and test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.1, random_state=42)

# Convertation data to PyTorch tensors
train_features = torch.tensor(X_train, dtype=torch.float32)
test_features = torch.tensor(X_test, dtype=torch.float32)
train_targets = torch.tensor(y_train, dtype=torch.float32)
test_targets = torch.tensor(y_test, dtype=torch.float32)    

# Creating DataLoader
train_data = TensorDataset(train_features, train_targets)
test_data = TensorDataset(test_features, test_targets)

train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=False)

In [9]:
# Create a Neural Network class with flexible settings
class FlexibleRegressionNN(nn.Module):
    def __init__(self, input_size: int, hidden_layers_sizes: list, dropout_rate: float) -> None:
        super(FlexibleRegressionNN, self).__init__()
        layers = list()

        # Define first and n-1 hidden layers
        neuron_num = input_size
        for size in hidden_layers_sizes:
            layers.append(nn.Linear(neuron_num, size))
            layers.append(nn.BatchNorm1d(size))
            layers.append(nn.LeakyReLU())
            layers.append(nn.Dropout(dropout_rate))
            neuron_num = size

        layers.append(nn.Linear(neuron_num, 1))
        self.model = nn.Sequential(*layers)

    # Define forward loop for NN
    def forward(self, x):
        return self.model(x)

In [7]:
# Define function for calculation RMSE
def root_mean_squared_error(y_true, y_pred):
    mse = torch.mean((y_true - y_pred) ** 2)
    rmse = torch.sqrt(mse)
    return rmse.item()

In [11]:
# Creating function for training and evaluating the model
def train_and_evaluate_model(model, train_loader, test_loader, criterion, optimizer, epochs):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)
        
        model.eval()
        with torch.no_grad():
            test_loss = 0.0
            for inputs, targets in test_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                test_loss += loss.item()
            test_loss /= len(test_loader)
        
        print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')
    
    return test_loss

# Creating function for selection of best hyperparameters 
def objective(trial):
    # Defining hyperparameters
    num_hidden_layers = trial.suggest_int('num_hidden_layers', 1, 10)
    hidden_layer_sizes = [
        trial.suggest_int(f'layer_{i}_size', 32, 256)
        for i in range(num_hidden_layers)
    ]
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    epochs = trial.suggest_int('epochs', 5, 100)
    input_size = 14

    # Creating a model and defining of optimizer
    model = FlexibleRegressionNN(input_size, hidden_layer_sizes, dropout_rate)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    # Dataloader
    train_data = torch.randn(1000, input_size), torch.randn(1000, 1)
    test_data = torch.randn(200, input_size), torch.randn(200, 1)
    train_loader = DataLoader(TensorDataset(*train_data), batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(TensorDataset(*test_data), batch_size=batch_size, shuffle=False)

    # Training launch and return the test loss
    return train_and_evaluate_model(model, train_loader, test_loader, criterion, optimizer, epochs)

# Settings of searching by Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=230)

# Getting the best hyperparameters
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2024-12-12 21:19:26,161] A new study created in memory with name: no-name-ac2130e2-85eb-4162-b2c6-ecf66212d007
/var/folders/qq/gt96f0sx7nn6wsr9d8n13r2w0000gn/T/ipykernel_75395/355664794.py:36: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
/var/folders/qq/gt96f0sx7nn6wsr9d8n13r2w0000gn/T/ipykernel_75395/355664794.py:37: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


Epoch 1/28, Train Loss: 1.1940, Test Loss: 0.9746
Epoch 2/28, Train Loss: 1.1643, Test Loss: 0.9689
Epoch 3/28, Train Loss: 1.0901, Test Loss: 0.9567
Epoch 4/28, Train Loss: 1.1472, Test Loss: 0.9652
Epoch 5/28, Train Loss: 1.1082, Test Loss: 0.9660
Epoch 6/28, Train Loss: 1.0822, Test Loss: 0.9656
Epoch 7/28, Train Loss: 1.1038, Test Loss: 0.9675
Epoch 8/28, Train Loss: 1.0927, Test Loss: 0.9720
Epoch 9/28, Train Loss: 1.0901, Test Loss: 0.9757
Epoch 10/28, Train Loss: 1.0947, Test Loss: 0.9691
Epoch 11/28, Train Loss: 1.0956, Test Loss: 0.9669
Epoch 12/28, Train Loss: 1.0626, Test Loss: 0.9673
Epoch 13/28, Train Loss: 1.0591, Test Loss: 0.9669
Epoch 14/28, Train Loss: 1.1217, Test Loss: 0.9678
Epoch 15/28, Train Loss: 1.0437, Test Loss: 0.9703
Epoch 16/28, Train Loss: 1.1080, Test Loss: 0.9781
Epoch 17/28, Train Loss: 1.0720, Test Loss: 0.9725
Epoch 18/28, Train Loss: 1.0879, Test Loss: 0.9723
Epoch 19/28, Train Loss: 1.0672, Test Loss: 0.9682
Epoch 20/28, Train Loss: 1.0814, Test Lo

[I 2024-12-12 21:19:30,666] Trial 0 finished with value: 0.9749857783317566 and parameters: {'num_hidden_layers': 10, 'layer_0_size': 80, 'layer_1_size': 77, 'layer_2_size': 170, 'layer_3_size': 57, 'layer_4_size': 130, 'layer_5_size': 227, 'layer_6_size': 142, 'layer_7_size': 252, 'layer_8_size': 198, 'layer_9_size': 210, 'dropout_rate': 0.3109885954578083, 'learning_rate': 0.00040078470478524845, 'batch_size': 64, 'epochs': 28}. Best is trial 0 with value: 0.9749857783317566.


Epoch 27/28, Train Loss: 1.0472, Test Loss: 0.9831
Epoch 28/28, Train Loss: 1.0578, Test Loss: 0.9750
Epoch 1/46, Train Loss: 1.3499, Test Loss: 1.0102
Epoch 2/46, Train Loss: 1.2096, Test Loss: 1.0040
Epoch 3/46, Train Loss: 1.2793, Test Loss: 0.9982
Epoch 4/46, Train Loss: 1.2223, Test Loss: 0.9897
Epoch 5/46, Train Loss: 1.2735, Test Loss: 0.9872
Epoch 6/46, Train Loss: 1.1963, Test Loss: 0.9896
Epoch 7/46, Train Loss: 1.2237, Test Loss: 0.9912
Epoch 8/46, Train Loss: 1.1875, Test Loss: 0.9907
Epoch 9/46, Train Loss: 1.1138, Test Loss: 0.9916
Epoch 10/46, Train Loss: 1.1934, Test Loss: 0.9869
Epoch 11/46, Train Loss: 1.1899, Test Loss: 0.9921
Epoch 12/46, Train Loss: 1.1377, Test Loss: 0.9925
Epoch 13/46, Train Loss: 1.1007, Test Loss: 0.9941
Epoch 14/46, Train Loss: 1.1316, Test Loss: 1.0003
Epoch 15/46, Train Loss: 1.1229, Test Loss: 0.9906
Epoch 16/46, Train Loss: 1.1273, Test Loss: 0.9934
Epoch 17/46, Train Loss: 1.1028, Test Loss: 0.9939
Epoch 18/46, Train Loss: 1.1624, Test Lo

[I 2024-12-12 21:19:36,132] Trial 1 finished with value: 0.9977798972811017 and parameters: {'num_hidden_layers': 5, 'layer_0_size': 156, 'layer_1_size': 59, 'layer_2_size': 220, 'layer_3_size': 199, 'layer_4_size': 229, 'dropout_rate': 0.39268877513854294, 'learning_rate': 9.41677046519703e-05, 'batch_size': 32, 'epochs': 46}. Best is trial 0 with value: 0.9749857783317566.


...

Epoch 71/72, Train Loss: 1.0680, Test Loss: 1.0912
Epoch 72/72, Train Loss: 1.1375, Test Loss: 1.0876
Epoch 1/16, Train Loss: 1.2129, Test Loss: 0.9642
Epoch 2/16, Train Loss: 1.2120, Test Loss: 0.9601
Epoch 3/16, Train Loss: 1.1841, Test Loss: 0.9561
Epoch 4/16, Train Loss: 1.1621, Test Loss: 0.9536
Epoch 5/16, Train Loss: 1.1422, Test Loss: 0.9526
Epoch 6/16, Train Loss: 1.1480, Test Loss: 0.9515
Epoch 7/16, Train Loss: 1.1333, Test Loss: 0.9540
Epoch 8/16, Train Loss: 1.1418, Test Loss: 0.9527
Epoch 9/16, Train Loss: 1.1320, Test Loss: 0.9531
Epoch 10/16, Train Loss: 1.1596, Test Loss: 0.9523
Epoch 11/16, Train Loss: 1.1385, Test Loss: 0.9538
Epoch 12/16, Train Loss: 1.1330, Test Loss: 0.9540


[I 2024-12-12 21:35:04,283] Trial 228 finished with value: 0.9584275782108307 and parameters: {'num_hidden_layers': 6, 'layer_0_size': 113, 'layer_1_size': 168, 'layer_2_size': 194, 'layer_3_size': 83, 'layer_4_size': 223, 'layer_5_size': 112, 'dropout_rate': 0.38624942706439397, 'learning_rate': 9.183171210035325e-05, 'batch_size': 128, 'epochs': 16}. Best is trial 44 with value: 0.7144440561532974.


Epoch 13/16, Train Loss: 1.1296, Test Loss: 0.9548
Epoch 14/16, Train Loss: 1.1186, Test Loss: 0.9568
Epoch 15/16, Train Loss: 1.1072, Test Loss: 0.9580
Epoch 16/16, Train Loss: 1.1204, Test Loss: 0.9584
Epoch 1/49, Train Loss: 1.2154, Test Loss: 1.0713
Epoch 2/49, Train Loss: 1.2604, Test Loss: 1.0632
Epoch 3/49, Train Loss: 1.2156, Test Loss: 1.0469
Epoch 4/49, Train Loss: 1.1784, Test Loss: 1.0476
Epoch 5/49, Train Loss: 1.2519, Test Loss: 1.0497
Epoch 6/49, Train Loss: 1.1826, Test Loss: 1.0478
Epoch 7/49, Train Loss: 1.1788, Test Loss: 1.0393
Epoch 8/49, Train Loss: 1.1298, Test Loss: 1.0391
Epoch 9/49, Train Loss: 1.1532, Test Loss: 1.0410
Epoch 10/49, Train Loss: 1.1873, Test Loss: 1.0315
Epoch 11/49, Train Loss: 1.1631, Test Loss: 1.0337
Epoch 12/49, Train Loss: 1.1104, Test Loss: 1.0328
Epoch 13/49, Train Loss: 1.1464, Test Loss: 1.0322
Epoch 14/49, Train Loss: 1.1175, Test Loss: 1.0318
Epoch 15/49, Train Loss: 1.1331, Test Loss: 1.0325
Epoch 16/49, Train Loss: 1.1175, Test Lo

[I 2024-12-12 21:35:08,212] Trial 229 finished with value: 1.0299981236457825 and parameters: {'num_hidden_layers': 7, 'layer_0_size': 133, 'layer_1_size': 178, 'layer_2_size': 221, 'layer_3_size': 42, 'layer_4_size': 188, 'layer_5_size': 215, 'layer_6_size': 174, 'dropout_rate': 0.4867337985879555, 'learning_rate': 0.00013808463208838165, 'batch_size': 64, 'epochs': 49}. Best is trial 44 with value: 0.7144440561532974.


Epoch 49/49, Train Loss: 1.0735, Test Loss: 1.0300
Best trial:
  Value: 0.7144440561532974
  Params:
    num_hidden_layers: 6
    layer_0_size: 133
    layer_1_size: 148
    layer_2_size: 182
    layer_3_size: 121
    layer_4_size: 235
    layer_5_size: 131
    dropout_rate: 0.3744673729438832
    learning_rate: 2.9805716870829137e-05
    batch_size: 64
    epochs: 99
